# Setup

In [ ]:
exp_directory = 'results'
agent_name = 'DeepQAgent'
game_name = 'Breakout'
render_mode='rgb_array'

### Global Modules

In [ ]:
import os
import pandas as pd
%matplotlib inline

### Local Modules

In [ ]:
import base
from src.agents import DeepQAgent, A3CAgent
from src.util import JupyterCallback
from src.environment.atari import build_atari_environment

## Constants

In [ ]:
agents = {
    DeepQAgent.__name__: DeepQAgent,
    A3CAgent.__name__: A3CAgent,
}

In [ ]:
exp_directory = '{}/{}/{}'.format(exp_directory, game_name, agent_name)
if not os.path.exists(exp_directory):
    os.makedirs(exp_directory)
exp_directory

# Environment

In [ ]:
env = build_atari_environment(game_name)

In [ ]:
env.observation_space

In [ ]:
env.action_space

# Agent

In [ ]:
agent = agents[agent_name](env, render_mode=render_mode)
agent

## Initial

In [ ]:
initial = agent.play()
initial = pd.Series(initial)
initial

In [ ]:
initial.to_csv('{}/initial.csv'.format(exp_directory))

In [ ]:
initial.describe()

In [ ]:
initial.hist()

## Training

In [ ]:
agent.observe()

In [ ]:
callback = JupyterCallback()
agent.train(callback=callback)

In [ ]:
# save the training results
scores = pd.Series(callback.scores)
scores.to_csv('{}/scores.csv'.format(exp_directory))
losses = pd.Series(callback.losses)
losses.to_csv('{}/losses.csv'.format(exp_directory))

## Final

In [ ]:
final = agent.play()
final = pd.Series(final)
final

In [ ]:
final.to_csv('{}/final.csv'.format(exp_directory))

In [ ]:
final.describe()

In [ ]:
final.hist()

## Saving Weights

In [ ]:
agent.model.save_weights('{}/weights.h5'.format(exp_directory), overwrite=True)